## Load dependencies and csv from part 1

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# get api key from environment variables
g_key = os.environ.get('VACATION_API_KEY')

# read csv from part 1 folder
part1_weather_df = pd.read_csv("../WeatherPy/part1.csv")
print(len(part1_weather_df))

556


In [2]:
# authenticate with google maps
gmaps.configure(api_key=str(g_key))

# heat map requires three variables: lat, lng, and weight
locations = part1_weather_df[['Lat', 'Lng']]
weights = part1_weather_df['Humidity']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# adjusting heat map to better maintain weight when zoomed
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 10

## Remove cities with less than ideal weather parameters.

In [8]:
# drop cities from DataFrame that do not have 'ideal' weather
best_weather_df = part1_weather_df.drop(part1_weather_df[
        (part1_weather_df["Max Temps"] > 80) |
        (part1_weather_df["Max Temps"] < 70) |
        (part1_weather_df["Cloudiness"] > 5) |
        (part1_weather_df["Wind Speed"] > 5)].index)

print(len(best_weather_df))

8


In [ ]:
# base url to use "Nearby Search" from Places API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# search radius is 5000 meters from city coordinates
radius = "&radius=5000"
# only look for hotels
type_param = "&type=lodging"
api_key = "&key="+str(g_key)
hotel_names = []

# for loop to build API call based on city coordinates
# get first hotel result, and add to hotel_names list
for index, row in best_weather_df.iterrows():
    # get lat,lng for each row in best_weather_df
    location = "location="+str(row['Lat'])+","+str(row['Lng'])
    # build url based on location
    URL = base_url+location+radius+type_param+api_key
    # http request
    response = requests.get(URL)
    # get data in json format
    data = response.json()
    # get the hotel name of the first result and add it to hotel_names list
    hotel_names.append(data['results'][0]['name'])

# add list of hotels to df
best_weather_df['Hotel Nearby'] = hotel_names